In [2]:
# 1. Build a Flask app that scrapes data from multiple websites and displays it on your site.
# You can try to scrap websites like youtube , amazon and show data on output pages and deploy it on cloud
# platform .


from flask import Flask, render_template
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)

def scrape_youtube():
    url = 'https://www.youtube.com'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Example: scrape trending video titles
    titles = [title.text for title in soup.find_all('h3', class_='title-and-badge')]
    return titles[:5]  # Return first 5 titles

@app.route('/')
def index():
    youtube_titles = scrape_youtube()
    return render_template('index.html', titles=youtube_titles)

if __name__ == '__main__':
    app.run(debug=True)

    
HTML Template (templates/index.html):
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Scraped Data</title>
</head>
<body>
    <h1>YouTube Trending Videos</h1>
    <ul>
        {% for title in titles %}
            <li>{{ title }}</li>
        {% endfor %}
    </ul>
</body>
</html>


In [ ]:
# 2. Create a Flask app that consumes data from external APIs and displays it to users.
# Try to find an public API which will give you a data and based on that call it and deploy it on cloud platform

from flask import Flask, render_template
import requests

app = Flask(__name__)

@app.route('/')
def index():
    url = 'https://api.github.com/repos/python/cpython'
    response = requests.get(url)
    data = response.json()
    repo_name = data['name']
    stars = data['stargazers_count']
    return render_template('index.html', repo_name=repo_name, stars=stars)

if __name__ == '__main__':
    app.run(debug=True)

    
HTML Template (templates/index.html):
    
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>GitHub Repo Data</title>
</head>
<body>
    <h1>Repository: {{ repo_name }}</h1>
    <p>Stars: {{ stars }}</p>
</body>
</html>


In [ ]:
# 3. Implement OAuth2 authentication to allow users to log in using their Google or Facebook accounts.
from flask import Flask, redirect, url_for, session
from flask_oauthlib.client import OAuth

app = Flask(__name__)
app.secret_key = 'random_secret_key'
oauth = OAuth(app)

google = oauth.remote_app(
    'google',
    consumer_key='YOUR_GOOGLE_CLIENT_ID',
    consumer_secret='YOUR_GOOGLE_CLIENT_SECRET',
    request_token_params={
        'scope': 'email',
    },
    base_url='https://www.googleapis.com/oauth2/v1/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://accounts.google.com/o/oauth2/token',
    authorize_url='https://accounts.google.com/o/oauth2/auth',
)

@app.route('/')
def index():
    return 'Welcome to the Flask OAuth2 Example'

@app.route('/login')
def login():
    return google.authorize(callback=url_for('authorized', _external=True))

@app.route('/logout')
def logout():
    session.pop('google_token')
    return redirect(url_for('index'))

@app.route('/login/authorized')
def authorized():
    response = google.authorized_response()
    session['google_token'] = (response['access_token'], '')
    user_info = google.get('userinfo')
    return f'Logged in as: {user_info.data["email"]}'

@google.tokengetter
def get_google_oauth2_token():
    return session.get('google_token')

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# 4. Develop a recommendation system using Flask that suggests content to users based on their preferences.
from flask import Flask, render_template, request

app = Flask(__name__)

# Sample data for the recommendation system
items = {
    'action': ['Movie 1', 'Movie 2', 'Movie 3'],
    'comedy': ['Movie 4', 'Movie 5', 'Movie 6'],
    'drama': ['Movie 7', 'Movie 8', 'Movie 9'],
}

@app.route('/', methods=['GET', 'POST'])
def index():
    recommendations = []
    if request.method == 'POST':
        genre = request.form.get('genre')
        recommendations = items.get(genre, [])
    return render_template('index.html', recommendations=recommendations)

if __name__ == '__main__':
    app.run(debug=True)

    
HTML Template for Recommendation System (templates/index.html):
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Recommendation System</title>
</head>
<body>
    <h1>Movie Recommendation System</h1>
    <form method="POST">
        <label for="genre">Choose a genre:</label>
        <select name="genre" id="genre">
            <option value="action">Action</option>
            <option value="comedy">Comedy</option>
            <option value="drama">Drama</option>
        </select>
        <input type="submit" value="Get Recommendations">
    </form>
    <h2>Recommendations:</h2>
    <ul>
        {% for movie in recommendations %}
            <li>{{ movie }}</li>
        {% endfor %}
    </ul>
</body>
</html>
